In [6]:
import requests
import torch
from transformers import BlipForConditionalGeneration, BlipProcessor
from diffusers import DDIMScheduler, DDIMInverseScheduler, StableDiffusionPix2PixZeroPipeline
from PIL import Image
import numpy as np
import pickle
import glob
from pathlib import Path

def download(embedding_url, local_filepath):
    r = requests.get(embedding_url)
    with open(local_filepath, "wb") as f:
        f.write(r.content)

In [4]:
captioner_id = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(captioner_id)
model = BlipForConditionalGeneration.from_pretrained(captioner_id, torch_dtype=torch.float16, low_cpu_mem_usage=True)


Widget Javascript not detected.  It may not be installed or enabled properly.


OSError: Can't load image processor for 'Salesforce/blip-image-captioning-base'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'Salesforce/blip-image-captioning-base' is the correct path to a directory containing a preprocessor_config.json file

In [3]:
sd_model_ckpt = "CompVis/stable-diffusion-v1-4"
pipeline = StableDiffusionPix2PixZeroPipeline.from_pretrained(
    sd_model_ckpt,
    caption_generator=model,
    caption_processor=processor,
    torch_dtype=torch.float16,
    safety_checker=None,
)
pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
pipeline.inverse_scheduler = DDIMInverseScheduler.from_config(pipeline.scheduler.config)
pipeline.enable_model_cpu_offload()

NameError: name 'model' is not defined

In [ ]:
def get_caption(image):
    caption = pipeline.generate_caption(image)
    return caption

In [ ]:
def get_reconstructed_image(raw_image):
    caption = get_caption(raw_image)
    generator = torch.manual_seed(0)
    inv_latents = pipeline.invert(caption, image=raw_image, generator=generator).latents
    source_embeds = pipeline.get_embeds(caption, batch_size=2)
    pred_img = pipeline(
            caption,
            source_embeds=source_embeds,
            target_embeds=source_embeds,
            num_inference_steps=50,
            cross_attention_guidance_amount=0.15,
            generator=generator,
            latents=inv_latents,
            negative_prompt=caption,
        ).images[0]
    
    return pred_img, caption, inv_latents

In [ ]:
def calculate_psnr(original_image, reconstructed_image):
    # Convert images to numpy arrays
    original_array = np.array(original_image)
    reconstructed_array = np.array(reconstructed_image)

    # Calculate mean squared error
    mse = np.mean((original_array - reconstructed_array) ** 2)

    # Maximum possible pixel value
    max_pixel_value = 255  # Assuming 8-bit images

    # Calculate PSNR
    psnr = 20 * np.log10(max_pixel_value / np.sqrt(mse))

    return psnr

In [ ]:
def calculate_bpp(data_path, image_size):
    # Get the file size of the image in bytes
    file_size_bytes = os.path.getsize(data_path)

    # Convert file size to bits
    file_size_bits = file_size_bytes * 8

    width, height = image_size

    # Calculate total number of pixels
    total_pixels = width * height

    # Calculate bits per pixel
    bpp = file_size_bits / total_pixels

    return bpp

In [ ]:
img_files =sorted(glob.glob('Jockey/*.png'))
psnr_values =[]
bpp_values = []
for file in img_files:
    frame = Image.open(file).convert("RGB").resize((512, 512))
    pred , cap, latent = get_reconstructed_image(frame)
    psnr = calculate_psnr(frame,pred)   
    psnr_values.append(psnr)
    pred.save('compressed/'+ Path(file).stem+ '.png')
    output_name = 'compressed/'+ Path(file).stem + '.pkl'
    with open(output_name, 'wb') as f:
        pickle.dump({'caption': cap, 'tensor': latent}, f)
    bpp = calculate_bpp(output_name,(512,512))
    bpp_values.append(bpp)


In [ ]:
def get_diff_caption(ref_img, img, ref_caption):


